In [76]:
import sys
from tqdm import tqdm
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, Dataset, RandomSampler, SequentialSampler
import pickle
import random
import numpy as np
import pandas as pd
from generate_dataloaders import *
from sklearn.model_selection import train_test_split
from ast import literal_eval

from transformers import (
    BertModel,
    BertTokenizer
)

from transformers import glue_convert_examples_to_features as convert_examples_to_features
from transformers.data.processors.utils import InputExample
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [77]:
datasets = pd.read_excel("./data/master_df_labeled.xlsx",index_col=0)
datasets.review = datasets.review.apply(literal_eval)

### Check for uncaught 11's, update accordingly

In [80]:
#check for weird 11's
datasets[datasets.true_pos==11]

,review,flagged_word,flagged_index,problematic,lemmatized,true_pos


In [79]:
#replace them!
datasets.true_pos[datasets.true_pos==11]=datasets.true_pos[datasets.true_pos==11].replace(11,1)

/anaconda3/envs/capstone/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [81]:
datasets=datasets[datasets['review'].apply(lambda x: len(x)<=30)]

In [82]:
# Divide ground_truth(test) and train/val dataset
labeled = datasets.loc[datasets['true_pos'].isin([1,0])]
unlabeled = datasets[datasets.true_pos==-1]

In [83]:
#train_labeled
train_labeled_dataset, remains_label_dataset = train_test_split(labeled, test_size=0.2, stratify=labeled['true_pos'])

In [84]:
#val_dataset, test_dataset
val_dataset, test_dataset = train_test_split(remains_label_dataset, test_size=0.5, stratify=remains_label_dataset['true_pos'])

In [85]:
val_idx = val_dataset.index.tolist()
test_idx = test_dataset.index.tolist()

remove_idx = val_idx + test_idx

In [86]:
#train_dataset - Atul doesn't really need
train_dataset = datasets.drop(remove_idx, axis=0)

In [87]:
train_label_idx = train_labeled_dataset.index.tolist()
#train_unlabeled
train_unlabeled_dataset = train_dataset.drop(train_label_idx,axis=0)

In [88]:
all_datasets = {"train_labeled_dataset":train_labeled_dataset,
               "val_dataset": val_dataset,
               "test_dataset":test_dataset,
               "train_unlabeled_dataset":train_unlabeled_dataset}

In [91]:
def preprocessing(all_df):
  preprocess_dict = {}
  for key, df in all_df.items():
    list_of_dicts=[]
    for row in df.iterrows():
      review = row[1].review
      idx = row[1].flagged_index
      text_a = " ".join(review[:idx+1])
      text_b = " ".join(review[idx:])
      temp_obj=InputExample(guid=str(row[0]), text_a=text_a, text_b=text_b, label=str(row[1].true_pos))
      list_of_dicts.append(temp_obj)
    preprocess_dict[key] = list_of_dicts
  return preprocess_dict

In [92]:
#Make dataset into format expected for creating bert dataloaders
preprocess_dict = preprocessing(all_datasets)

In [45]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

100%|██████████| 213450/213450 [00:00<00:00, 6268900.29B/s]


In [93]:
preprocess_dict["train_labeled_dataset"]

[{
   "guid": "58596",
   "label": "0",
   "text_a": "thank you and hopefully we 'll work together",
   "text_b": "together again in the future ."
 }, {
   "guid": "21892",
   "label": "1",
   "text_a": "he 's experienced",
   "text_b": "experienced and talented ."
 }, {
   "guid": "58257",
   "label": "1",
   "text_a": "libby is a fantastic asset to have as part of your team , she is knowledge , honest",
   "text_b": "honest and hard working ."
 }, {
   "guid": "16565",
   "label": "1",
   "text_a": "logo was creative",
   "text_b": "creative and he turned it around quickly"
 }, {
   "guid": "24229",
   "label": "1",
   "text_a": "the job was difficult , but he was able to understand",
   "text_b": "understand what i wanted and came up with a very original design ."
 }, {
   "guid": "13400",
   "label": "1",
   "text_a": "excellent",
   "text_b": "excellent quality of work"
 }, {
   "guid": "131757",
   "label": "1",
   "text_a": "excellent writing ability , judy also has a great",
  

In [59]:
def to_tensored(dataset):
  features = convert_examples_to_features(dataset,
                                          tokenizer,
                                          label_list=['0','1','-1'],
                                          max_length=40,
                                          output_mode='classification',
                                          pad_on_left=False,
                                          pad_token=tokenizer.pad_token_id,
                                          pad_token_segment_id=0)
  tensored_dataset = TensorDataset(torch.tensor([f.input_ids for f in features], dtype=torch.long), 
                                torch.tensor([f.attention_mask for f in features], dtype=torch.long), 
                                torch.tensor([f.token_type_ids for f in features], dtype=torch.long), 
                                torch.tensor([f.label for f in features], dtype=torch.long))
  return tensored_dataset

In [94]:
bert_train_labeled_dataloader = DataLoader(to_tensored(preprocess_dict["train_labeled_dataset"]), batch_size=32)
bert_train_unlabeled_dataloader = DataLoader(to_tensored(preprocess_dict["train_unlabeled_dataset"]), batch_size=32)
bert_val_dataloader = DataLoader(to_tensored(preprocess_dict["val_dataset"]), batch_size=32)
bert_test_dataloader = DataLoader(to_tensored(preprocess_dict["test_dataset"]), batch_size=32)


In [96]:
path = os.getcwd()
data_dir = path + '/data/'

with open(data_dir + "bert_train_labeled_dataloader.p","wb") as f:
  pickle.dump(bert_train_labeled_dataloader, f)

with open(data_dir + "bert_train_unlabeled_dataloader.p","wb") as f:
  pickle.dump(bert_train_unlabeled_dataloader, f)

with open(data_dir + "bert_val_dataloader.p","wb") as f:
  pickle.dump(bert_val_dataloader, f)

with open(data_dir + "bert_test_dataloader.p","wb") as f:
  pickle.dump(bert_test_dataloader, f)
